# Step 2: Build the Tokenizer from Scratch 🔤

In this notebook, we'll implement **Byte Pair Encoding (BPE)** from scratch to understand how LLMs convert text into numbers.

## What We'll Learn

1. **Why Tokenization?** - Why LLMs can't just use characters or words
2. **Byte Pair Encoding** - The algorithm that powers GPT-2, GPT-3, GPT-4
3. **Loading the Vocabulary** - Understanding `vocab.json` and `merges.txt`
4. **Encoding** - Convert text → token IDs
5. **Decoding** - Convert token IDs → text

---

## Why Do We Need Tokenization?

Neural networks work with **numbers**, not text. We need to convert:

```
"Hello world" → [15496, 995] → Model → [284, 262, ...] → "to the ..."
```

### Option 1: Characters ❌

```
"Hello" → ['H', 'e', 'l', 'l', 'o'] → [7, 4, 11, 11, 14]
```

**Problems:**
- Very long sequences (each character = 1 token)
- Model must learn spelling from scratch
- Wastes capacity on low-level patterns

### Option 2: Whole Words ❌

```
"tokenization" → ID 47832
"tokenize" → ID 12543
"tokens" → ID 8921
```

**Problems:**
- Huge vocabulary (every word form needs its own ID)
- Unknown words can't be handled
- "tokenization" and "tokenize" share no representation!

### Option 3: Subwords ✅ (BPE)

```
"tokenization" → ["token", "ization"] → [30001, 1634]
"tokenize" → ["token", "ize"] → [30001, 1096]
```

**Benefits:**
- Compact vocabulary (GPT-2 uses 50,257 tokens)
- Common words = single tokens
- Rare words = broken into meaningful subwords
- Shared subwords share learned representations!


---

## How BPE Works: The Core Idea

BPE is elegantly simple:

1. **Start** with all individual bytes (256 base tokens)
2. **Count** which pairs of adjacent tokens appear most frequently
3. **Merge** the most frequent pair into a new token
4. **Repeat** until you reach your target vocabulary size

### Training Example (How the vocabulary was created)

Training corpus: `"low lower lowest lowering"`

```
Step 1: Start with characters
  ['l', 'o', 'w', ' ', 'l', 'o', 'w', 'e', 'r', ...]

Step 2: Count pairs
  ('l', 'o') appears 4 times  ← most frequent!
  ('o', 'w') appears 4 times
  ('e', 'r') appears 3 times
  ...

Step 3: Merge ('l', 'o') → 'lo'
  ['lo', 'w', ' ', 'lo', 'w', 'e', 'r', ...]

Step 4: Count again, merge again...
  ('lo', 'w') → 'low'
  ['low', ' ', 'low', 'e', 'r', ...]

Continue until vocabulary size reached!
```

The **merge rules** are saved to `merges.txt` during training. At inference time, we just apply these rules!


---

## Setup and Imports


In [68]:
import json
import regex  # Note: 'regex' not 're' - needed for Unicode categories
from functools import lru_cache


---

## Understanding GPT-2's Byte Encoding

GPT-2 uses a clever trick: instead of working with raw bytes (0-255), it maps bytes to **printable Unicode characters**. This makes the vocabulary human-readable.

### The Byte-to-Character Mapping

```
Byte 32 (space)    → 'Ġ' (special character for space)
Byte 33 ('!')      → '!'
Byte 65 ('A')      → 'A'
Byte 10 (newline)  → 'Ċ' (special character)
```

The `Ġ` character you see in `merges.txt` represents a space at the start of a word. This is how GPT-2 distinguishes:
- `"the"` at the start of text (no Ġ)
- `" the"` with a leading space (becomes `"Ġthe"`)


In [69]:
def bytes_to_unicode():
    """
    Create a mapping from bytes (0-255) to Unicode characters.
    
    GPT-2 uses this to make the vocabulary human-readable.
    Non-printable bytes get mapped to unused Unicode characters.
    
    Returns:
        dict: Mapping from byte value (int) to Unicode character (str)
    """
    # Printable ASCII characters (these stay the same)
    # Ranges: ! to ~ (33-126), plus some extended Latin
    printable = list(range(ord("!"), ord("~") + 1))  # 33-126
    printable += list(range(ord("¡"), ord("¬") + 1))  # 161-172
    printable += list(range(ord("®"), ord("ÿ") + 1))  # 174-255
    
    # Start with printable bytes mapping to themselves
    byte_values = printable[:]
    unicode_chars = printable[:]
    
    # For non-printable bytes (0-32, 127-160, 173), 
    # map them to Unicode characters starting at 256
    n = 0
    for b in range(256):
        if b not in printable:
            byte_values.append(b)
            unicode_chars.append(256 + n)  # Use chars above Latin-1
            n += 1
    
    # Convert to dict: byte_value -> Unicode character
    return {b: chr(c) for b, c in zip(byte_values, unicode_chars)}


# Create the mapping
byte_encoder = bytes_to_unicode()
byte_decoder = {v: k for k, v in byte_encoder.items()}  # Reverse mapping

print("Byte to Unicode Mapping Examples:")
print(f"  Space (byte 32)   → '{byte_encoder[32]}' (Ġ represents leading space)")
print(f"  'A' (byte 65)     → '{byte_encoder[65]}'")
print(f"  'a' (byte 97)     → '{byte_encoder[97]}'")
print(f"  Newline (byte 10) → '{byte_encoder[10]}' (Ċ represents newline)")
print(f"  Tab (byte 9)      → '{byte_encoder[9]}'")
print(f"\nTotal mappings: {len(byte_encoder)}")


Byte to Unicode Mapping Examples:
  Space (byte 32)   → 'Ġ' (Ġ represents leading space)
  'A' (byte 65)     → 'A'
  'a' (byte 97)     → 'a'
  Newline (byte 10) → 'Ċ' (Ċ represents newline)
  Tab (byte 9)      → 'ĉ'

Total mappings: 256


---

## Loading the Vocabulary Files

GPT-2's tokenizer needs two files:

1. **`vocab.json`**: Maps token strings to token IDs
2. **`merges.txt`**: Lists the BPE merge rules in order (earlier = higher priority)


In [70]:
# Load vocabulary (token string -> token ID)
with open("models/gpt2/vocab.json", "r", encoding="utf-8") as f:
    vocab = json.load(f)

# Create reverse mapping (token ID -> token string)
vocab_inverse = {v: k for k, v in vocab.items()}

print(f"Vocabulary size: {len(vocab):,} tokens")
print(f"\nSome example tokens:")
print(f"  '!' → {vocab['!']}")
print(f"  'hello' → {vocab.get('hello', 'NOT FOUND')}")
print(f"  'Ġhello' → {vocab.get('Ġhello', 'NOT FOUND')}")
print(f"  'Ġthe' → {vocab['Ġthe']}")
print(f"  'ĠHello' → {vocab.get('ĠHello', 'NOT FOUND')}")


Vocabulary size: 50,257 tokens

Some example tokens:
  '!' → 0
  'hello' → 31373
  'Ġhello' → 23748
  'Ġthe' → 262
  'ĠHello' → 18435


In [71]:
# Load BPE merge rules
with open("models/gpt2/merges.txt", "r", encoding="utf-8") as f:
    merges_text = f.read()

# Parse the merges file
# First line is version info, skip it
merge_lines = merges_text.split("\n")[1:]  # Skip "#version: 0.2"
merge_lines = [line for line in merge_lines if line]  # Remove empty lines

# Create merge rules: (token1, token2) -> priority (lower = merge first)
bpe_merges = {}
for i, line in enumerate(merge_lines):
    parts = line.split(" ")
    if len(parts) == 2:
        bpe_merges[tuple(parts)] = i

print(f"Number of merge rules: {len(bpe_merges):,}")
print(f"\nFirst 10 merge rules (highest priority):")
for i, line in enumerate(merge_lines[:10]):
    a, b = line.split(" ")
    merged = a + b
    print(f"  {i}: '{a}' + '{b}' → '{merged}'")


Number of merge rules: 50,000

First 10 merge rules (highest priority):
  0: 'Ġ' + 't' → 'Ġt'
  1: 'Ġ' + 'a' → 'Ġa'
  2: 'h' + 'e' → 'he'
  3: 'i' + 'n' → 'in'
  4: 'r' + 'e' → 're'
  5: 'o' + 'n' → 'on'
  6: 'Ġt' + 'he' → 'Ġthe'
  7: 'e' + 'r' → 'er'
  8: 'Ġ' + 's' → 'Ġs'
  9: 'a' + 't' → 'at'


---

## Understanding the Merge Rules

Let's decode what these merge rules mean:

| Rule | Token 1 | Token 2 | Result | Meaning |
|------|---------|---------|--------|---------|
| 0 | `Ġ` | `t` | `Ġt` | Space + t |
| 1 | `Ġ` | `a` | `Ġa` | Space + a |
| 2 | `h` | `e` | `he` | "he" is common! |
| 3 | `i` | `n` | `in` | "in" is common! |
| 7 | `Ġt` | `he` | `Ġthe` | Space + "the" |

The order matters! Lower number = higher priority = gets merged first.


In [72]:
# Let's look at some interesting merge rules
print("Some interesting merge rules:")
print()

# Find merges that create common words
interesting_merges = [
    ("Ġt", "he"),   # → "Ġthe" (" the")
    ("Ġa", "nd"),   # → "Ġand" (" and")
    ("Ġo", "f"),    # → "Ġof" (" of")
    ("Ġi", "s"),    # → "Ġis" (" is")
]

for a, b in interesting_merges:
    if (a, b) in bpe_merges:
        priority = bpe_merges[(a, b)]
        merged = a + b
        # Decode for display
        display = merged.replace("Ġ", "⎵")  # Show space as visible char
        print(f"  Priority {priority:5}: '{a}' + '{b}' → '{merged}' (displayed: '{display}')")


Some interesting merge rules:

  Priority     6: 'Ġt' + 'he' → 'Ġthe' (displayed: '⎵the')
  Priority    30: 'Ġo' + 'f' → 'Ġof' (displayed: '⎵of')


---

## The BPE Algorithm

Now let's implement BPE! The algorithm has these steps:

1. **Convert text to bytes** (using our byte encoder)
2. **Split into initial tokens** (each byte is one token)
3. **Repeatedly merge** the highest-priority pair
4. **Stop** when no more merges apply


In [73]:
def get_pairs(tokens):
    """
    Get all adjacent pairs from a list of tokens.
    
    Example:
        ['h', 'e', 'l', 'l', 'o'] → {('h', 'e'), ('e', 'l'), ('l', 'l'), ('l', 'o')}
    """
    pairs = set()
    for i in range(len(tokens) - 1):
        pairs.add((tokens[i], tokens[i + 1]))
    return pairs


# Test it
test_tokens = list("hello")
print(f"Tokens: {test_tokens}")
print(f"Pairs: {get_pairs(test_tokens)}")


Tokens: ['h', 'e', 'l', 'l', 'o']
Pairs: {('e', 'l'), ('l', 'o'), ('h', 'e'), ('l', 'l')}


In [74]:
def bpe(token, verbose=False):
    """
    Apply BPE merges to a single word/token.
    
    Args:
        token: A string (already converted to byte-encoded characters)
        verbose: If True, print each merge step
        
    Returns:
        tuple: The token split into BPE subwords
    """
    if len(token) <= 1:
        return tuple(token)
    
    # Start with each character as a separate token
    word = list(token)
    
    if verbose:
        print(f"Starting with: {word}")
    
    while True:
        # Get all adjacent pairs in current tokenization
        pairs = get_pairs(word)
        
        if not pairs:
            break
        
        # Find the pair with lowest merge priority (= should merge first)
        # If a pair isn't in our merge rules, give it infinity priority
        best_pair = min(pairs, key=lambda p: bpe_merges.get(p, float('inf')))
        
        # If best pair isn't in our merge rules, we're done
        if best_pair not in bpe_merges:
            break
        
        # Merge this pair everywhere in the word
        first, second = best_pair
        new_word = []
        i = 0
        while i < len(word):
            # Look for the pair to merge
            if i < len(word) - 1 and word[i] == first and word[i + 1] == second:
                # Merge!
                new_word.append(first + second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        
        if verbose:
            priority = bpe_merges[best_pair]
            print(f"  Merge '{first}' + '{second}' (priority {priority}) → {new_word}")
        
        word = new_word
        
        # If we're down to a single token, we're done
        if len(word) == 1:
            break
    
    return tuple(word)


In [75]:
# Let's test BPE on a simple word!
# First, convert "the" to byte-encoded form
def text_to_bytes(text):
    """Convert text to GPT-2's byte-encoded string."""
    return ''.join(byte_encoder[b] for b in text.encode('utf-8'))

# Test on " the" (note the leading space!)
test_word = " the"
byte_encoded = text_to_bytes(test_word)

print(f"Original text: '{test_word}'")
print(f"Byte-encoded: '{byte_encoded}'")
print(f"  (The Ġ character represents the leading space)")
print()

# Apply BPE with verbose output
result = bpe(byte_encoded, verbose=True)
print(f"\nFinal BPE tokens: {result}")


Original text: ' the'
Byte-encoded: 'Ġthe'
  (The Ġ character represents the leading space)

Starting with: ['Ġ', 't', 'h', 'e']
  Merge 'Ġ' + 't' (priority 0) → ['Ġt', 'h', 'e']
  Merge 'h' + 'e' (priority 2) → ['Ġt', 'he']
  Merge 'Ġt' + 'he' (priority 6) → ['Ġthe']

Final BPE tokens: ('Ġthe',)


In [76]:
# Let's try a longer word: "tokenization"
test_word = " tokenization"
byte_encoded = text_to_bytes(test_word)

print(f"Original text: '{test_word}'")
print(f"Byte-encoded: '{byte_encoded}'")
print()

result = bpe(byte_encoded, verbose=True)
print(f"\nFinal BPE tokens: {result}")


Original text: ' tokenization'
Byte-encoded: 'Ġtokenization'

Starting with: ['Ġ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n']
  Merge 'Ġ' + 't' (priority 0) → ['Ġt', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n']
  Merge 'o' + 'n' (priority 5) → ['Ġt', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'on']
  Merge 'a' + 't' (priority 9) → ['Ġt', 'o', 'k', 'e', 'n', 'i', 'z', 'at', 'i', 'on']
  Merge 'e' + 'n' (priority 12) → ['Ġt', 'o', 'k', 'en', 'i', 'z', 'at', 'i', 'on']
  Merge 'Ġt' + 'o' (priority 28) → ['Ġto', 'k', 'en', 'i', 'z', 'at', 'i', 'on']
  Merge 'i' + 'on' (priority 39) → ['Ġto', 'k', 'en', 'i', 'z', 'at', 'ion']
  Merge 'at' + 'ion' (priority 85) → ['Ġto', 'k', 'en', 'i', 'z', 'ation']
  Merge 'i' + 'z' (priority 272) → ['Ġto', 'k', 'en', 'iz', 'ation']
  Merge 'iz' + 'ation' (priority 1378) → ['Ġto', 'k', 'en', 'ization']
  Merge 'k' + 'en' (priority 3208) → ['Ġto', 'ken', 'ization']
  Merge 'Ġto' + 'ken' (priority 10985) → ['Ġtoken', 'ization']



---

## Pre-Tokenization: Splitting Text into Words

Before applying BPE, GPT-2 first splits text into "words" using a regex pattern. This ensures:
- Punctuation is separated from words
- Numbers are treated specially
- Contractions like "don't" stay together


In [77]:
# GPT-2's pre-tokenization pattern
# This splits text into chunks before BPE is applied to each chunk
GPT2_PATTERN = regex.compile(
    r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
)

# Let's understand this pattern:
# - 's|'t|'re|'ve|'m|'ll|'d  : English contractions
# - | ?\p{L}+                : Optional space + letters (words)
# - | ?\p{N}+                : Optional space + numbers  
# - | ?[^\s\p{L}\p{N}]+      : Optional space + punctuation
# - |\s+(?!\S)               : Trailing whitespace
# - |\s+                     : Other whitespace

test_text = "Hello, world! I'm learning about GPT-2's tokenizer. It uses 50257 tokens."
chunks = GPT2_PATTERN.findall(test_text)

print(f"Text: '{test_text}'")
print(f"\nPre-tokenized chunks ({len(chunks)} chunks):")
for i, chunk in enumerate(chunks):
    # Show spaces explicitly
    display = chunk.replace(' ', '⎵')
    print(f"  {i:2}: '{display}'")


Text: 'Hello, world! I'm learning about GPT-2's tokenizer. It uses 50257 tokens.'

Pre-tokenized chunks (19 chunks):
   0: 'Hello'
   1: ','
   2: '⎵world'
   3: '!'
   4: '⎵I'
   5: ''m'
   6: '⎵learning'
   7: '⎵about'
   8: '⎵GPT'
   9: '-'
  10: '2'
  11: ''s'
  12: '⎵tokenizer'
  13: '.'
  14: '⎵It'
  15: '⎵uses'
  16: '⎵50257'
  17: '⎵tokens'
  18: '.'


---

## Complete Encoder: Text → Token IDs

Now let's put it all together into a complete encoder!


In [78]:
# Cache BPE results for efficiency
bpe_cache = {}

def encode(text):
    """
    Encode text into GPT-2 token IDs.
    
    Steps:
    1. Split text into chunks using regex
    2. Convert each chunk to byte-encoded form
    3. Apply BPE to each chunk
    4. Look up token IDs in vocabulary
    
    Returns:
        list: Token IDs
    """
    token_ids = []
    
    # Step 1: Pre-tokenize using regex
    chunks = GPT2_PATTERN.findall(text)
    
    for chunk in chunks:
        # Step 2: Convert to byte-encoded string
        byte_encoded = text_to_bytes(chunk)
        
        # Step 3: Apply BPE (with caching)
        if byte_encoded not in bpe_cache:
            bpe_cache[byte_encoded] = bpe(byte_encoded)
        bpe_tokens = bpe_cache[byte_encoded]
        
        # Step 4: Look up token IDs
        for token in bpe_tokens:
            if token in vocab:
                token_ids.append(vocab[token])
            else:
                # This shouldn't happen with proper BPE, but handle gracefully
                print(f"Warning: Unknown token '{token}'")
    
    return token_ids


# Test it!
test_text = "Hello, world!"
token_ids = encode(test_text)
print(f"Text: '{test_text}'")
print(f"Token IDs: {token_ids}")
print(f"\nToken breakdown:")
for tid in token_ids:
    token_str = vocab_inverse[tid]
    display = token_str.replace("Ġ", "⎵")
    print(f"  {tid:6} → '{token_str}' (displayed: '{display}')")


Text: 'Hello, world!'
Token IDs: [15496, 11, 995, 0]

Token breakdown:
   15496 → 'Hello' (displayed: 'Hello')
      11 → ',' (displayed: ',')
     995 → 'Ġworld' (displayed: '⎵world')
       0 → '!' (displayed: '!')


---

## Decoder: Token IDs → Text

Decoding is simpler than encoding:
1. Look up each token ID → token string
2. Concatenate all token strings
3. Convert byte-encoded string back to text


In [79]:
def decode(token_ids):
    """
    Decode token IDs back into text.
    
    Steps:
    1. Look up each token ID → token string
    2. Concatenate all strings
    3. Convert from byte-encoded characters back to UTF-8 text
    
    Returns:
        str: Decoded text
    """
    # Step 1 & 2: Look up and concatenate token strings
    byte_encoded_text = ''.join(vocab_inverse[tid] for tid in token_ids)
    
    # Step 3: Convert byte-encoded characters back to bytes, then decode as UTF-8
    byte_values = [byte_decoder[c] for c in byte_encoded_text]
    text = bytes(byte_values).decode('utf-8', errors='replace')
    
    return text


# Test roundtrip: encode then decode
original = "Hello, world! How are you today?"
encoded = encode(original)
decoded = decode(encoded)

print(f"Original:  '{original}'")
print(f"Encoded:   {encoded}")
print(f"Decoded:   '{decoded}'")
print(f"\nRoundtrip successful: {original == decoded} ✓" if original == decoded else "\n❌ Roundtrip failed!")


Original:  'Hello, world! How are you today?'
Encoded:   [15496, 11, 995, 0, 1374, 389, 345, 1909, 30]
Decoded:   'Hello, world! How are you today?'

Roundtrip successful: True ✓


---

## Visualizing Tokenization

Let's create a nice visualization to see how different texts get tokenized!


In [80]:
def visualize_tokenization(text):
    """
    Visualize how text gets tokenized.
    
    Shows:
    - Each token with its ID
    - Token boundaries in the original text
    - Number of tokens used
    """
    token_ids = encode(text)
    
    print(f"┌{'─' * 70}┐")
    print(f"│ Text: {text[:62]:<62} │")
    print(f"├{'─' * 70}┤")
    print(f"│ Tokens: {len(token_ids):<60} │")
    print(f"├{'─' * 70}┤")
    
    # Show each token
    token_display = []
    for tid in token_ids:
        token_str = vocab_inverse[tid]
        # Decode the token for display
        try:
            decoded = bytes([byte_decoder[c] for c in token_str]).decode('utf-8')
        except:
            decoded = token_str
        display = repr(decoded)[1:-1]  # Remove quotes, keep escapes visible
        token_display.append((tid, display))
    
    # Print tokens in rows
    row = "│ "
    for tid, display in token_display:
        token_repr = f"[{tid}:'{display}']"
        if len(row) + len(token_repr) + 2 > 71:
            print(f"{row:<71}│")
            row = "│ "
        row += token_repr + " "
    if len(row) > 2:
        print(f"{row:<71}│")
    
    print(f"└{'─' * 70}┘")


# Try some examples!
examples = [
    "Hello, world!",
    "The quick brown fox jumps over the lazy dog.",
    "GPT-2 uses 50257 tokens in its vocabulary.",
    "Supercalifragilisticexpialidocious",
]

for example in examples:
    visualize_tokenization(example)
    print()


┌──────────────────────────────────────────────────────────────────────┐
│ Text: Hello, world!                                                  │
├──────────────────────────────────────────────────────────────────────┤
│ Tokens: 4                                                            │
├──────────────────────────────────────────────────────────────────────┤
│ [15496:'Hello'] [11:','] [995:' world'] [0:'!']                      │
└──────────────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────────────┐
│ Text: The quick brown fox jumps over the lazy dog.                   │
├──────────────────────────────────────────────────────────────────────┤
│ Tokens: 10                                                           │
├──────────────────────────────────────────────────────────────────────┤
│ [464:'The'] [2068:' quick'] [7586:' brown'] [21831:' fox']           │
│ [18045:' jumps'] [625:' over'] [262:' the'] [169

---

## Exploring Token Efficiency

Let's see how efficient tokenization is for different types of text!


In [81]:
def analyze_tokenization(text):
    """Analyze tokenization efficiency."""
    token_ids = encode(text)
    chars = len(text)
    tokens = len(token_ids)
    ratio = chars / tokens if tokens > 0 else 0
    
    return {
        "text": text[:50] + "..." if len(text) > 50 else text,
        "chars": chars,
        "tokens": tokens,
        "chars_per_token": ratio
    }


# Test different types of text
test_cases = [
    ("Common English", "The quick brown fox jumps over the lazy dog. It was a beautiful sunny day."),
    ("Technical code", "def fibonacci(n): return n if n <= 1 else fibonacci(n-1) + fibonacci(n-2)"),
    ("Numbers", "The year 2024 had 365 days, and 2025 will have 365 too. Pi is 3.14159."),
    ("Rare words", "Pneumonoultramicroscopicsilicovolcanoconiosis is a lung disease."),
    ("Repetition", "the the the the the the the the the the"),
    ("All caps", "HELLO WORLD THIS IS ALL UPPERCASE TEXT"),
]

print(f"{'Type':<20} {'Chars':>6} {'Tokens':>7} {'Chars/Token':>12}")
print("-" * 50)
for name, text in test_cases:
    result = analyze_tokenization(text)
    print(f"{name:<20} {result['chars']:>6} {result['tokens']:>7} {result['chars_per_token']:>12.2f}")


Type                  Chars  Tokens  Chars/Token
--------------------------------------------------
Common English           74      17         4.35
Technical code           73      31         2.35
Numbers                  70      21         3.33
Rare words               64      20         3.20
Repetition               39      10         3.90
All caps                 38      12         3.17


---

## Special Tokens

GPT-2 has some special tokens worth knowing about:


In [82]:
# Find some interesting tokens
print("Special and interesting tokens:\n")

# The end-of-text token
eot_token = "<|endoftext|>"
if eot_token in vocab:
    print(f"  End of text token: '{eot_token}' → {vocab[eot_token]}")

print("\nSome common word tokens:")
common_words = ["Ġthe", "Ġa", "Ġand", "Ġto", "Ġof", "Ġin", "Ġis", "Ġthat", "Ġit"]
for word in common_words:
    if word in vocab:
        display = word.replace("Ġ", "⎵")
        print(f"  '{display}' → {vocab[word]}")

print("\nSome single-letter tokens (first 10):")
for i, char in enumerate("abcdefghij"):
    if char in vocab:
        print(f"  '{char}' → {vocab[char]}")


Special and interesting tokens:

  End of text token: '<|endoftext|>' → 50256

Some common word tokens:
  '⎵the' → 262
  '⎵a' → 257
  '⎵and' → 290
  '⎵to' → 284
  '⎵of' → 286
  '⎵in' → 287
  '⎵is' → 318
  '⎵that' → 326
  '⎵it' → 340

Some single-letter tokens (first 10):
  'a' → 64
  'b' → 65
  'c' → 66
  'd' → 67
  'e' → 68
  'f' → 69
  'g' → 70
  'h' → 71
  'i' → 72
  'j' → 73


---

## Putting It All Together: A Complete Tokenizer Class

Let's wrap everything into a clean, reusable class!


In [83]:
class GPT2Tokenizer:
    """
    A from-scratch implementation of GPT-2's BPE tokenizer.
    
    This implements the full tokenization pipeline:
    - Byte-level encoding
    - Regex-based pre-tokenization  
    - BPE merging
    - Vocabulary lookup
    """
    
    def __init__(self, vocab_path="models/gpt2/vocab.json", merges_path="models/gpt2/merges.txt"):
        # Load vocabulary
        with open(vocab_path, "r", encoding="utf-8") as f:
            self.vocab = json.load(f)
        self.vocab_inverse = {v: k for k, v in self.vocab.items()}
        
        # Load merge rules
        with open(merges_path, "r", encoding="utf-8") as f:
            merges_text = f.read()
        merge_lines = [line for line in merges_text.split("\n")[1:] if line]
        self.bpe_merges = {}
        for i, line in enumerate(merge_lines):
            parts = line.split(" ")
            if len(parts) == 2:
                self.bpe_merges[tuple(parts)] = i
        
        # Create byte encoder/decoder
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v: k for k, v in self.byte_encoder.items()}
        
        # Pre-tokenization pattern
        self.pattern = regex.compile(
            r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
        )
        
        # BPE cache
        self.cache = {}
    
    def _get_pairs(self, word):
        """Get all adjacent pairs in a word."""
        return set((word[i], word[i + 1]) for i in range(len(word) - 1))
    
    def _bpe(self, token):
        """Apply BPE to a single token."""
        if token in self.cache:
            return self.cache[token]
        
        word = list(token)
        
        while len(word) > 1:
            pairs = self._get_pairs(word)
            if not pairs:
                break
            
            best_pair = min(pairs, key=lambda p: self.bpe_merges.get(p, float('inf')))
            if best_pair not in self.bpe_merges:
                break
            
            first, second = best_pair
            new_word = []
            i = 0
            while i < len(word):
                if i < len(word) - 1 and word[i] == first and word[i + 1] == second:
                    new_word.append(first + second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            word = new_word
        
        result = tuple(word)
        self.cache[token] = result
        return result
    
    def encode(self, text):
        """Convert text to token IDs."""
        token_ids = []
        
        for chunk in self.pattern.findall(text):
            # Convert to byte-encoded string
            byte_encoded = ''.join(self.byte_encoder[b] for b in chunk.encode('utf-8'))
            
            # Apply BPE
            bpe_tokens = self._bpe(byte_encoded)
            
            # Look up token IDs
            for token in bpe_tokens:
                if token in self.vocab:
                    token_ids.append(self.vocab[token])
        
        return token_ids
    
    def decode(self, token_ids):
        """Convert token IDs back to text."""
        text = ''.join(self.vocab_inverse[tid] for tid in token_ids)
        byte_values = [self.byte_decoder[c] for c in text]
        return bytes(byte_values).decode('utf-8', errors='replace')
    
    def __len__(self):
        """Return vocabulary size."""
        return len(self.vocab)


# Create our tokenizer!
tokenizer = GPT2Tokenizer()
print(f"✅ GPT2Tokenizer created with {len(tokenizer):,} tokens")


✅ GPT2Tokenizer created with 50,257 tokens


In [84]:
# Test the complete tokenizer
test_texts = [
    "Hello, world!",
    "The quick brown fox jumps over the lazy dog.",
    "GPT-2 is a language model with 124M parameters.",
    "Let's test contractions: I'm, you're, they've, won't.",
]

print("Testing GPT2Tokenizer:")
print("=" * 60)

for text in test_texts:
    encoded = tokenizer.encode(text)
    decoded = tokenizer.decode(encoded)
    
    print(f"\nOriginal: '{text}'")
    print(f"Encoded:  {encoded}")
    print(f"Decoded:  '{decoded}'")
    print(f"Match: {'✓' if text == decoded else '✗'}")


Testing GPT2Tokenizer:

Original: 'Hello, world!'
Encoded:  [15496, 11, 995, 0]
Decoded:  'Hello, world!'
Match: ✓

Original: 'The quick brown fox jumps over the lazy dog.'
Encoded:  [464, 2068, 7586, 21831, 18045, 625, 262, 16931, 3290, 13]
Decoded:  'The quick brown fox jumps over the lazy dog.'
Match: ✓

Original: 'GPT-2 is a language model with 124M parameters.'
Encoded:  [38, 11571, 12, 17, 318, 257, 3303, 2746, 351, 19755, 44, 10007, 13]
Decoded:  'GPT-2 is a language model with 124M parameters.'
Match: ✓

Original: 'Let's test contractions: I'm, you're, they've, won't.'
Encoded:  [5756, 338, 1332, 2775, 507, 25, 314, 1101, 11, 345, 821, 11, 484, 1053, 11, 1839, 470, 13]
Decoded:  'Let's test contractions: I'm, you're, they've, won't.'
Match: ✓


---

## Step 2 Complete!

We've successfully built a **complete BPE tokenizer from scratch**!

### What We Learned:

1. **Why BPE?** - Balances vocabulary size with handling rare words
2. **Byte Encoding** - GPT-2 maps bytes to printable Unicode characters
3. **Pre-tokenization** - Regex splits text before BPE
4. **BPE Algorithm** - Greedily merge highest-priority pairs
5. **Vocabulary Lookup** - Convert BPE tokens to IDs

### Key Insights:

- `G` with dot = space prefix - Distinguishes "the" from " the"
- Common words = single tokens - "the", "and", "is" are each one token
- Rare words = multiple subwords - "tokenization" becomes "token" + "ization"
- Vocabulary size: 50,257 - A sweet spot for English + code + symbols

### Next Step:

Now that we can convert text to numbers, we will learn how to **embed these numbers into vectors** that the model can process! We will explore token embeddings and positional encodings in Step 3.
